In [26]:
import sys
sys.path.append('../src/')

import numpy as np
import scipy
from scipy.sparse import eye, kron, csr_matrix
from Hamiltonians import Hamiltonians
from tqdm import tqdm
from numba import jit
import cython

from functools import wraps
import time

In [10]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [31]:
%%cython 

import sys
sys.path.append('../src/')

import numpy as np
import scipy
from scipy.sparse import eye, kron, csr_matrix
from Hamiltonians import Hamiltonians
from tqdm import tqdm
from numba import jit
import cython

from functools import wraps
import time

cpdef psi_dot_psi(list psi1, list psi2):
    cdef double x = 0.
    for i in range(psi1.shape[0]):
        for j in range(psi2.shape[1]):
            x += psi1[i, j] * psi2[i, j]
    return x

In [84]:
%%cython

import sys
sys.path.append('../src/')

import numpy as np
import scipy
from scipy.sparse import eye, kron, csr_matrix
from Hamiltonians import Hamiltonians
from tqdm import tqdm
from numba import jit
import cython

from functools import wraps
import time

from libcpp cimport bool
cimport numpy as np

np.import_array()
DTYPE = np.double

ctypedef np.double_t DTYPE_t

cpdef psi_dot_psi(np.ndarray psi1, np.ndarray psi2):
    cdef double x = 0.
    for i in range(psi1.shape[0]):
        for j in range(psi2.shape[1]):
            x += psi1[i, j] * psi2[i, j]
    return x

cpdef lanczos(m, seed, int maxiter, double tol, bint use_seed=False, bint force_maxiter=False):
    cdef np.ndarray x1 = seed
    cdef np.ndarray x2 = seed
    cdef np.ndarray gs = seed
    cdef np.ndarray a = np.zeros(100)
    cdef np.ndarray b = np.zeros(100)
    cdef np.ndarray z = np.zeros((100, 100))
    cdef list lvectors = []
    cdef int control_max = maxiter;
    cdef int e0 = 9999
    # cdef bint force_maxiter = force_maxiter

    # if (maxiter == -1):
    #     force_maxiter = False

    if (control_max == 0):
        gs = 1
        maxiter = 1
        return (e0, gs)

    x1[:, :] = 0
    x2[:, :] = 0
    gs[:, :] = 0
    a[:] = 0.0
    b[:] = 0.0
    if (use_seed):
        x1 = seed
    else:
        for i in range(x1.shape[0]):
            for j in range(x1.shape[1]):
                x1[i, j] = (2 * np.random.random() - 1.)

    #    x1[:,:] = 1
    b[0] = psi_dot_psi(x1, x1)
    b[0] = np.sqrt(b[0])
    x1 = x1 / b[0]
    x2[:] = 0
    b[0] = 1.

    e0 = 9999
    nmax = min(99, maxiter)

    for iter in range(1, nmax + 1):
        eini = e0
        if (b[iter - 1] != 0.):
            aux = x1
            x1 = -b[iter - 1] * x2
            x2 = aux / b[iter - 1]

        aux = m.product(x2)

        x1 = x1 + aux
        a[iter] = psi_dot_psi(x1, x2)
        x1 = x1 - x2 * a[iter]

        b[iter] = psi_dot_psi(x1, x1)
        b[iter] = np.sqrt(b[iter])
        lvectors.append(x2)
        #        print "Iter =",iter,a[iter],b[iter]
        z.resize((iter, iter), refcheck=False)
        z[:, :] = 0
        for i in range(0, iter - 1):
            z[i, i + 1] = b[i + 1]
            z[i + 1, i] = b[i + 1]
            z[i, i] = a[i + 1]
        z[iter - 1, iter - 1] = a[iter]
        d, v = np.linalg.eig(z)

        col = 0
        n = 0
        e0 = 9999
        for e in d:
            if (e < e0):
                print(e0, e)
                e0 = e
                col = n
            n += 1
        e0 = d[col]

        # print("Iter = ",iter," Ener = ",e0)
        if ((force_maxiter and iter >= control_max) or (
                iter >= gs.shape[0] * gs.shape[1] or iter == 99 or abs(b[iter]) < tol) or \
                ((not force_maxiter) and abs(eini - e0) <= tol)):
            # converged
            gs[:, :] = 0.
            for n in range(0, iter):
                gs += v[n, col] * lvectors[n]

            # print("E0 = ", e0)
            maxiter = iter
            return (e0, gs)  # We return with ground states energy

    return (e0, gs)

In file included from /Users/garethjones/.ipython/cython/_cython_magic_aeeff2e91e5660e3a935b3dca2816563.c:684:
In file included from /Users/garethjones/opt/anaconda3/lib/python3.9/site-packages/numpy/core/include/numpy/arrayobject.h:4:
In file included from /Users/garethjones/opt/anaconda3/lib/python3.9/site-packages/numpy/core/include/numpy/ndarrayobject.h:12:
In file included from /Users/garethjones/opt/anaconda3/lib/python3.9/site-packages/numpy/core/include/numpy/ndarraytypes.h:1944:
/Users/garethjones/opt/anaconda3/lib/python3.9/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: "Using deprecated NumPy API, disable it with "          "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-W#warnings]
#warning "Using deprecated NumPy API, disable it with " \
 ^
1 warning generated.


In [77]:
class DMRGHamiltonians(Hamiltonians):
    def __init__(self):
        super().__init__()
        self.sz = np.matrix(self.sz.toarray())
        self.splus = np.matrix(self.splus.toarray())

    def heisenberg_interaction(self, H, side='left', size=0, dim=0, splusRL=0, szRL=0):
        if side == 'left':
            H[size] = np.kron(H[size - 1], np.eye(2)) + \
                      np.kron(szRL[size - 1], self.sz) + \
                      0.5 * np.kron(splusRL[size - 1], self.splus.transpose()) + \
                      0.5 * np.kron(splusRL[size - 1].transpose(), self.splus)
        elif side == 'right':
            H[size] = np.kron(np.eye(2), H[size - 1]) + \
                      np.kron(self.sz, szRL[size - 1]) + \
                      0.5 * np.kron(self.splus.transpose(), splusRL[size - 1]) + \
                      0.5 * np.kron(self.splus, splusRL[size - 1].transpose())
        return H[size]

In [82]:
class Position:
    LEFT, RIGHT = range(2)


class DMRG(object):

    def __init__(self, _nsites, _nsweeps, _n_states_to_keep):

        self.nsites = _nsites
        self.n_sweeps = _nsweeps
        self.n_states_to_keep = _n_states_to_keep
        self.nstates = 2
        self.dim_l = 0  # dimension of left block
        self.dim_r = 0  # dimension of right block
        self.left_size = 0  # number of sites in left block
        self.right_size = 0  # number of sites in right block

        self.sz0 = .5 * np.matrix('-1,0;0,1')
        self.splus0 = np.matrix('0,1;0,0')

        self.HL = []  # left block Hamiltonian
        self.HR = []  # right block Hamiltonian
        self.szL = []  # left block Sz
        self.szR = []  # right block Sz
        self.splusL = []  # left block S+
        self.splusR = []  # right block S+

        for i in range(self.nsites):
            self.HL.append(np.zeros(shape=(2, 2)))
            self.HR.append(np.zeros(shape=(2, 2)))
            self.szL.append(self.sz0)
            self.szR.append(self.sz0)
            self.splusL.append(self.splus0)
            self.splusR.append(self.splus0)

        self.psi = np.zeros((2, 2), dtype=np.float)  # ground state wave function
        self.rho = np.zeros((2, 2), dtype=np.float)  # density matrix

        self.energy = 0
        self.error = 0

    def build_block_left(self, it):
        self.left_size = it
        self.dim_l = self.HL[self.left_size-1].shape[0]

        # Enlarge left block
        ham = DMRGHamiltonians()
        # print("Appending to HL index {:d}".format(self.left_size))
        self.HL[self.left_size] = ham.heisenberg_interaction(self.HL, side='left', size=self.left_size,
                                                             dim=self.dim_l, splusRL=self.splusL, szRL=self.szL)

        self.splusL[self.left_size] = np.kron(np.eye(self.dim_l), self.splus0)
        self.szL[self.left_size] = np.kron(np.eye(self.dim_l), self.sz0)

    def build_block_right(self, it):
        self.right_size = it
        self.dim_r = self.HR[self.right_size - 1].shape[0]
        ham = DMRGHamiltonians()
        self.HR[self.right_size] = ham.heisenberg_interaction(self.HR, side='right', size=self.right_size,
                                                             dim=self.dim_r, splusRL=self.splusR, szRL=self.szR)
        self.splusR[self.right_size] = np.kron(self.splus0, np.eye(self.dim_r))
        self.szR[self.right_size] = np.kron(self.sz0, np.eye(self.dim_r))

    def ground_state(self):
        self.dim_l = self.HL[self.left_size].shape[0]
        self.dim_r = self.HR[self.right_size].shape[0]
        self.psi.resize((self.dim_l, self.dim_r))
        maxiter = self.dim_l*self.dim_r
        # print("\n")
        # print(self)
        # print(maxiter)
        (self.energy, self.psi) = lanczos(self, self.psi, maxiter, 1.e-7)
        # (self.energy, self.psi) = np.linalg.eigh(self.psi)

    def density_matrix(self, position):
        if position == Position.LEFT:
            self.rho = np.dot(self.psi, self.psi.transpose())
        else:
            self.rho = np.dot(self.psi.transpose(), self.psi)

    def truncate(self, position, m):
        # diagonalize rho
        rho_eig, rho_evec = scipy.sparse.linalg.eigsh(self.rho)
        self.nstates = m
        rho_evec = np.real(rho_evec)
        rho_eig = np.real(rho_eig)

        # calculate the truncation error for a given number of states m
        # Reorder eigenvectors and trucate
        index = np.argsort(rho_eig)
        # for e in index:
        #     print("RHO EIGENVALUE ", rho_eig[e])
        error = 0.
        if (m < rho_eig.shape[0]):
            for i in range(index.shape[0] - m):
                error += rho_eig[index[i]]
        # print("Truncation error = ", error)

        aux = np.copy(rho_evec)
        if (self.rho.shape[0] > m):
            aux.resize((aux.shape[0], m), refcheck=False)
            n = 0
            for i in range(index.shape[0] - 1, index.shape[0] - 1 - m, -1):
                aux[:, n] = rho_evec[:, index[i]]
                n += 1
        rho_evec = aux

        #        rho_evec = np.eye(self.rho.shape[0])

        # perform transformation:
        U = rho_evec.transpose()
        if (position == Position.LEFT):
            aux2 = np.dot(self.HL[self.left_size], rho_evec)
            self.HL[self.left_size] = np.dot(U, aux2)
            aux2 = np.dot(self.splusL[self.left_size], rho_evec)
            self.splusL[self.left_size] = np.dot(U, aux2)
            aux2 = np.dot(self.szL[self.left_size], rho_evec)
            self.szL[self.left_size] = np.dot(U, aux2)
        else:
            aux2 = np.dot(self.HR[self.right_size], rho_evec)
            self.HR[self.right_size] = np.dot(U, aux2)
            aux2 = np.dot(self.splusR[self.right_size], rho_evec)
            self.splusR[self.right_size] = np.dot(U, aux2)
            aux2 = np.dot(self.szR[self.right_size], rho_evec)
            self.szR[self.right_size] = np.dot(U, aux2)

    def product(self, psi):
        npsi = np.dot(self.HL[self.left_size], psi)
        npsi += np.dot(psi, self.HR[self.right_size].transpose())

        # Sz.Sz
        tmat = np.dot(psi, self.szR[self.right_size].transpose())
        npsi += np.dot(self.szL[self.left_size], tmat)
        # S+.S-
        tmat = np.dot(psi, self.splusR[self.right_size]) * 0.5
        npsi += np.dot(self.splusL[self.left_size], tmat)
        # S-.S+
        tmat = np.dot(psi, self.splusR[self.right_size].transpose()) * 0.5
        npsi += np.dot(self.splusL[self.left_size].transpose(), tmat)

        return npsi

    def _infinite_dmrg(self):
        for i in tqdm(range(1, self.n_sweeps)):
            for iter in range(1, int(self.nsites / 2)):  # do infinite size dmrg for warmup
                # print("WARMUP ITERATION ", iter, S.dim_l, S.dim_r)
                # Create HL and HR by adding the single sites to the two blocks
                self.build_block_left(iter)
                self.build_block_right(iter)

                # find smallest eigenvalue and eigenvector
                self.ground_state()

                # Calculate density matrix
                self.density_matrix(Position.LEFT)

                # Truncate
                self.truncate(Position.LEFT, self.n_states_to_keep)

                # Reflect
                self.density_matrix(Position.RIGHT)
                self.truncate(Position.RIGHT, self.n_states_to_keep)

    def _finite_dmrg(self):
        first_iter = int(self.nsites / 2)
        for i in tqdm(range(1, self.n_sweeps)):
            for sweep in range(1, self.n_sweeps):
                for i in range(first_iter, self.nsites - 3):
                    # print("LEFT-TO-RIGHT ITERATION ", iter, S.dim_l, S.dim_r)
                    # Create HL and HR by adding the single sites to the two blocks
                    self.build_block_left(i)
                    self.build_block_right(self.nsites - i - 2)
                    # find smallest eigenvalue and eigenvector
                    self.ground_state()
                    # Calculate density matrix
                    self.density_matrix(Position.LEFT)
                    # Truncate
                    self.truncate(Position.LEFT, self.n_states_to_keep)
                first_iter = 1
                for i in range(first_iter, self.nsites - 3):
                    # print("RIGHT-TO-LEFT ITERATION ", iter, S.dim_l, S.dim_r)
                    # Create HL and HR by adding the single sites to the two blocks
                    self.build_block_right(i)
                    self.build_block_left(self.nsites - i - 2)
                    # find smallest eigenvalue and eigenvector
                    self.ground_state()
                    # Calculate density matrix
                    self.density_matrix(Position.RIGHT)
                    # Truncate
                    self.truncate(Position.RIGHT, self.n_states_to_keep)

    # @jit(forceobj=True)
    def get_density(self):
        self._infinite_dmrg()
        self._finite_dmrg()
        return self.rho

In [85]:
def run_dmrg_heisenberg():
    nsites = 10
    n_states_to_keep = 10
    n_sweeps = 6
    S = DMRG(nsites, n_sweeps, n_states_to_keep)
    rho = S.get_density()
    print(rho)


if __name__ == '__main__':
    tstrt = time.time()
    run_dmrg_heisenberg()
    tstp = time.time()

    time_taken = tstp-tstrt
    print(time_taken)

/var/folders/lc/0pd94g0j0hx750lshn2s454r0000gn/T/ipykernel_13857/1836193933.py:36: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.psi = np.zeros((2, 2), dtype=np.float)  # ground state wave function
/var/folders/lc/0pd94g0j0hx750lshn2s454r0000gn/T/ipykernel_13857/1836193933.py:37: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.rho

9999 -0.08429181511418032
9999 0.6775499478753311
0 -0.9520892094810606
9999 -0.025686730889416247
9999 -1.183305281044624
9999 1.9645206418593502
1 -0.0669526041356505
0 -1.9614554835760272
9999 -0.1522308958362831
9999 -1.4266174488759586
9999 1.790927204976311
1 -0.2916128869003843
0 -2.15215849293447
9999 2.1213729149832465
2 0.6342519756701238
0 -1.196405687162201
-1 -2.779316374284571
9999 -1.4800636379303602
9999 -0.001092472987827886
0 -4.181355838248869
9999 0.6533232714726642
0 -2.284639327977055
-2 -5.9529135953496235
9999 1.1592921496321713
1 -0.9355948889761085
0 -4.026642683620609
-4 -7.426851582415859
9999 -7.964284098783176
9999 0.07550279707788692
9999 0.6290165019974747
0 -1.0299745455727158
9999 -1.3400394329031329
9999 -0.18377750887621347
9999 1.1093639218415188
1 -1.530451328057128
9999 -2.0806305971998693
9999 -2.3571229883486717
9999 -0.32335056642190013
9999 0.865316351871923
0 -1.7202499942979252
9999 1.4026352139456104
1 -0.4607325231751351
0 -2.4195306423827

100%|█████████████████████████████████████████████| 5/5 [00:00<00:00, 15.03it/s]


 -7.791839839627689
9999 -8.251963841036241
9999 1.8376194635511272
1 0.4252156600848058
0 -8.414345518087453
9999 -0.3638575192504566
9999 0.3078490044789382
0 -1.0428050158969078
9999 -1.4230704966296872
9999 -0.17268730102237043
9999 0.8108162932494775
0 -1.4494564464627402
9999 -2.100493972031408
9999 -2.2792772734436344
9999 -0.1395645819876387
9999 1.1156516226486957
1 -1.6317365740043157
9999 1.7754143010880732
1 -0.3113512829087558
0 -2.541598714528461
9999 -3.1210809578454075
9999 -3.388805513209844
9999 -1.5097655758062345
9999 0.017900301893289505
0 -4.433838788778195
9999 0.6031546195745512
0 -2.5779343069100435
-2 -6.443891636107376
9999 -7.725555104184041
9999 -8.16109170970652
9999 -8.306655615943281


  0%|                                                     | 0/5 [00:00<?, ?it/s]

...

9999 1.1831325226584395
1 -0.5286268963984199
0 -4.065600416240938
-4 -9.78348644263736
9999 -6.737644850687871
9999 -0.8530531813650715
0 -14.902419190018689
9999 -0.0959647225689757
0 -16.29434137341948
9999 -17.39164157523643
9999 0.7893906862721272
0 -2.085387021425495
-2 -17.852255669018408
9999 -15.58514640747997
9999 -1.9920014760132787
-1 -33.633216719402895
9999 -35.228917748027264
9999 0.2914218520232321
0 -9.408152112204021
-9 -36.455185425478454
9999 -36.64034306491588
9999 -5.219855206904589
9999 -0.6410616970630052
0 -14.363204775453251
9999 -15.806502803266296
9999 0.5273984003327654
0 -3.329883953152981
-3 -16.935756156352536
9999 -17.566820709412898
9999 1.0722444336758756
1 -0.8777145539432178
0 -5.085318388680773
-5 -12.12458424820646
-12 -18.34403246489492
9999 -18.680598247453897
9999 -2.1697773406876433
9999 -0.0744355381716435
0 -7.074755402502458
9999 -8.416331526280414
9999 0.9848466830911367
0 -1.3053766125747879
-1 -9.274706532480868
9999 -9.702890214483736
9

 80%|████████████████████████████████████         | 4/5 [00:03<00:00,  1.21it/s]

9999 -28.563597233873068
9999 0.0530274153712984
0 -6.4386654236713206
-6 -30.335258559965848
9999 -30.656451148284944
9999 -3.692521249502235
9999 0.7108245347758686
0 -12.342677548125508
9999 -13.89156594577519
9999 3.8802014928589177
3 -0.2858577338979399
0 -15.387423322869973
9999 4.387165692252426
4 0.22076672723211535
0 -15.825731161981837
9999 -0.3622478120691788
9999 1.6751569340966888
1 -4.361535539700821
9999 -6.936552122950571
9999 4.911931283724573
4 -7.724425285326376
9999 5.383194770011949
5 2.0539848849199225
2 -0.7489398670354542
0 -8.19618828059396
9999 -8.518667947519495
9999 -0.7820433010390027
9999 0.5538864490304745
0 -3.698415168689527
9999 -5.37563687332803
9999 -6.2659549087178315
9999 -6.574927686377714
9999 -1.4146047823213896
9999 0.5434927996266143
0 -6.033496944634731
9999 -7.469376874930074
9999 2.17725017876897
2 -0.3442667462243387
0 -8.211337326213334
9999 -8.52112503950974
9999 -5.37812457063467
9999 -0.4761174156994077
0 -13.870259633919321
9999 0.217

100%|█████████████████████████████████████████████| 5/5 [00:04<00:00,  1.21it/s]

9999 -1.128382199574652
9999 1.1170821559425912
1 -6.001270887344823
9999 -8.116262839096096
9999 4.322791606050509
4 0.21398739961963018
0 -9.027755991097019
9999 -9.63854417605955
9999 -4.785273151376095
9999 -0.486016773675372
0 -15.45151327883626
9999 0.11684679766047346
0 -17.831318066606578
9999 -19.088691996320783
9999 0.7660934660046423
0 -1.6001933660477627
-1 -9.40841671603228
-9 -19.79009034591089
[[ 4.48841316e-01  6.25450930e-02 -5.85863272e-02  1.33293224e-02
   1.08732315e-02 -9.48531047e-04  4.50215158e-01  6.47396939e-02
  -5.83194979e-02  1.59856871e-02  2.74478343e-04  4.60442837e-02
   4.79565451e-02 -1.38444843e-02  7.38759045e-03  3.01919509e-03
   7.01442717e-04  4.41611716e-02  4.61796106e-02 -1.45548224e-02]
 [ 6.25450930e-02  9.72534576e-03 -8.72568339e-03  1.96561267e-03
   1.63779440e-03 -3.62437348e-06  6.27348539e-02  9.80839547e-03
  -8.70156575e-03  2.55711479e-03  5.44277301e-04  6.22929000e-03
   6.76310331e-03 -1.74770818e-03  7.76090545e-04  4.121295

In [ ]:
(int m, double seed, int maxiter, double tol, bint use_seed=False, bint force_maxiter=False)

In [2]:
%%cython

a: cython.int = 0
for i in range(10):
    a += i
print(a)

45


In [3]:
%%cython
cpdef int test(int x):
    cdef int y = 1
    cdef int i
    for i in range(1, x+1):
        y *= i
    return y